# Project 1

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
players_info = pd.read_csv("People.csv")
players_info

In [ ]:
salary_data = pd.read_csv("Salaries.csv")
salary_data

In [ ]:
player_awards = pd.read_csv("AwardsPlayers.csv")
player_awards

In [ ]:
batting = pd.read_csv("Batting.csv")
batting.columns

In [ ]:
player_awards = player_awards[["playerID", "awardID", "yearID", "lgID"]]
players_info = players_info[["playerID", "weight", "height", "bats", "birthCountry", "nameFirst", "nameLast"]]
batting = batting[["playerID", "yearID", "lgID", "teamID", "H", "AB"]]
players_info = pd.merge(players_info, salary_data, on="playerID")
players_info = pd.merge(players_info, batting, on=["playerID", "lgID", "teamID", "yearID"])
players_info

## The most awarded Puerto Rican player

In [ ]:
# Narrow down to Puerto Rican players who have earned awards
awards = pd.merge(players_info, player_awards, on=["playerID", "lgID", "yearID"], how='outer')
PR_players = awards[awards.birthCountry == "P.R."]
PR_players = PR_players.dropna(subset="awardID")
# Run a count of awards won by each player and order by number of awards
PR_players_based_on_awards = PR_players.groupby(["playerID"])["awardID"].count().reset_index().sort_values(by="awardID", ascending=False)

In [ ]:
# Pick out the player with the most total awards and get data for awards won each year of their career
player_with_highest_number_of_awards = PR_players_based_on_awards['playerID'].iloc[0]
awards_per_year = PR_players[PR_players.playerID == player_with_highest_number_of_awards]
award_info = awards_per_year.groupby(["yearID"])["awardID"].count().reset_index()
# Find salary information for each year
salary_info = PR_players[PR_players.playerID == player_with_highest_number_of_awards].groupby(["yearID"])["salary"].first().reset_index()

# Plot findings in two separate ways
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

axs[0, 0].plot(award_info["yearID"], award_info["awardID"])
axs[0, 0].set_title('Award Number')
axs[0, 0].set_xticks(np.arange(award_info["yearID"].min(), award_info["yearID"].max() + 1, 1))
axs[0, 0].set_yticks(np.arange(award_info["awardID"].min(), award_info["awardID"].max() + 1, 1))
axs[0, 0].tick_params(axis='x', rotation=45)
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Number of awards')


axs[0, 1].plot(salary_info["yearID"], salary_info["salary"])
axs[0, 1].set_title('Salary')
axs[0, 1].set_xticks(np.arange(salary_info["yearID"].min(), salary_info["yearID"].max() + 1, 1))
axs[0, 1].tick_params(axis='x', rotation=45)
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Salary')



axs[1, 0].bar(award_info["yearID"], award_info["awardID"], color='green')  
axs[1, 0].set_title('Award Number')
axs[1, 0].set_xticks(np.arange(award_info["yearID"].min(), award_info["yearID"].max() + 1, 1))
axs[1, 0].tick_params(axis='x', rotation=45)
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Number of awards')



axs[1, 1].bar(salary_info["yearID"], salary_info["salary"], color='blue') 
axs[1, 1].set_title('Salary')
axs[1, 1].set_xticks(np.arange(salary_info["yearID"].min(), salary_info["yearID"].max() + 1, 1))
axs[1, 1].tick_params(axis='x', rotation=45)
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Salary')


plt.tight_layout()
plt.show()

In [ ]:

# Gather info and plot types of awards won each year
awards_of_highest = player_awards[player_awards['playerID'] == player_with_highest_number_of_awards]
sns.scatterplot(x="yearID", y="awardID", data=awards_of_highest)
plt.ylabel("Awards type")
plt.xlabel("Year")

plt.show()

## Impact of BMI on Batting Average

In [ ]:
# Gather necessary player info and drop players that may be missing necessary info
BMI_impact_on_run = players_info[["playerID", "teamID", "weight", "height", "H", "AB"]]
BMI_impact_on_run = BMI_impact_on_run.dropna(subset="H")
BMI_impact_on_run = BMI_impact_on_run.dropna(subset="AB")
BMI_impact_on_run = BMI_impact_on_run.dropna(subset="height")
BMI_impact_on_run = BMI_impact_on_run.dropna(subset="playerID")
BMI_impact_on_run = BMI_impact_on_run.dropna(subset="weight")

# Calculate BMIs, then for each, sum the hits and at-bats of players with that BMI and compute a batting average from these sums
BMI_impact_on_run['BMI'] = (BMI_impact_on_run.weight * 703) / (BMI_impact_on_run.height)**2
BMI_impact_on_run = BMI_impact_on_run.groupby('BMI').agg({"H": "sum", "AB": "sum"}).reset_index()
BMI_impact_on_run = BMI_impact_on_run[BMI_impact_on_run['AB'] > 20]
BMI_impact_on_run['battingAvg'] = (BMI_impact_on_run.H / BMI_impact_on_run.AB)
BMI_impact_on_run = BMI_impact_on_run.dropna(subset="battingAvg")

# Plot a linear regression to see if there is a relationship between the two
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 10))


plt.subplot(2, 1, 1)
sns.regplot(data=BMI_impact_on_run, x="BMI", y="battingAvg", scatter_kws={'s': 10})


# Plot a second one with players within the "normal range" of BMI
BMI_impact_on_run = BMI_impact_on_run[BMI_impact_on_run['BMI'] >= 18.5]
BMI_impact_on_run = BMI_impact_on_run[BMI_impact_on_run['BMI'] <= 24.9]

plt.subplot(2, 1, 2)
sns.regplot(data=BMI_impact_on_run, x="BMI", y="battingAvg", scatter_kws={'s': 10})

plt.ylabel("Batting average")

plt.tight_layout()

plt.show()

## Expected pay for each batting style in both leagues

In [ ]:
# Split the players into each league
national_league = players_info[(players_info.lgID == "NL") & (players_info.yearID > 2012)]
american_league = players_info[(players_info.lgID == "AL") & (players_info.yearID > 2012)]

In [ ]:
# Compute the average salary for each combination of batting style and league in recent years
bats_salary = players_info[players_info.yearID > 2012]
bats_salary = players_info.groupby(["bats", 'lgID'])["salary"].mean().reset_index()

# Plot the results
plt.ylabel("Salary")
plt.xlabel("Batting Style")

sns.barplot(x='bats', y='salary', hue='lgID', data=bats_salary)

## Money $pent on player salaries in each league

In [ ]:
# Split players between the two leagues
national_league = players_info[(players_info.lgID == "NL")]
american_league = players_info[(players_info.lgID == "AL")]

national_league_spent_money = national_league[["yearID" , "salary"]]
american_league_spent_money = american_league[["yearID" , "salary"]]

In [ ]:
# Compute the total salary expenditure per year in each league
national_league_spent_money = national_league_spent_money.groupby('yearID')['salary'].sum()
american_league_spent_money = american_league_spent_money.groupby('yearID')['salary'].sum()
national_league_spent_money = national_league_spent_money.reset_index()
american_league_spent_money = american_league_spent_money.reset_index()

In [ ]:
# plot the results
national_league_spent_money.columns = ["Year", "NA_spent_salary"]
american_league_spent_money.columns = ["Year", "AL_spent_salary"]
sns.lineplot(x="Year", y="NA_spent_salary", data=national_league_spent_money, color='red', label='NL')
sns.lineplot(x="Year", y="AL_spent_salary", data=american_league_spent_money, color='blue', label='AL')
plt.xlabel("Year")
plt.ylabel("The Amount of money spent per year")
plt.show()

## Expensive foreign-born players

In [ ]:
# Find players that played starting in 1990
among_years_1990_2014 = salary_data[(salary_data.yearID >= 1990) & (salary_data.yearID < 2014)]
among_years_1990_2014.reset_index()

In [ ]:
# Of the players, single out the ones that ever received a salary over $10,000,000 and who were born outside the US
expensive_players = players_info[players_info.salary > 10000000]
expensive_players_not_USA = expensive_players[expensive_players["birthCountry"] != "USA"]

In [ ]:
# Divide the players by birth country
unique_non_US_player = expensive_players_not_USA.drop_duplicates(subset=["playerID", "birthCountry"])
countries_count = unique_non_US_player.groupby('birthCountry')['playerID'].count().reset_index()
countries_count.columns = ['Countries', 'Count']
countries_count = countries_count.sort_values(ascending=False, by='Count')

In [ ]:
# Plot the findings
fig, axs = plt.subplots(1, 2, figsize=(25, 10))
axs[0].bar(countries_count["Countries"], countries_count["Count"])
axs[0].set_xlabel("Birth country")
axs[0].set_ylabel("Players paid a salary of over $10,000,000 at some point")
axs[0].tick_params(axis='x', rotation=45)
axs[1].pie(countries_count["Count"], labels=countries_count["Countries"])
plt.xlabel("Foreign players paid a salary of over $10,000,000 at some point")
plt.tight_layout()
plt.show()

## Expected starting salaries for foreigners

In [ ]:
players_info = pd.read_csv("People.csv")

# Find each player's starting year and then limit the data to the first year for players that started since 1990
starting_salary = salary_data.groupby('playerID')['yearID'].min().reset_index()
starting_salary = starting_salary[starting_salary.yearID >= 2010]
starting_salary.reset_index()
starting_salary = pd.merge(salary_data, starting_salary, on=['playerID', 'yearID'])
starting_salary.drop_duplicates('playerID')
# display(starting_salary.sort_values(ascending=False, by='salary'))
starting_salary = pd.merge(starting_salary, players_info, on='playerID')

# Find the average salary of players from each country in their starting year
# starting_salary = starting_salary[starting_salary['birthCountry'] != 'USA']
starting_salary = starting_salary.groupby('birthCountry')['salary'].mean().reset_index()
starting_salary = starting_salary.sort_values(ascending=False, by='salary')

# Show the results
display(starting_salary)
plt.figure(figsize=(10, 5))

plt.xlabel("Birth country")
plt.ylabel("Average starting salary")
sns.barplot(x='birthCountry', y='salary', data=starting_salary.head(10))